### lets talk about CHaching 
What is cache?



storing the data frame or a set of logics in memeroy so that we can reuse them down the oder with our wasting our resources

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [4]:
spark = (
    SparkSession
    .builder
    .config("spark.driver.memory","10g")
    .config("spark.sql.files.maxPartitionBytes", "268435456")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .master("local[*]")
    .getOrCreate()
)
sc= spark.sparkContext
sc.setLogLevel("ERROR")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/19 16:58:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!ls /Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data/data_skew/


customers.parquet    transactions.parquet


In [6]:
# Load both parquet datasets
df_customers = spark.read.parquet("/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data/data_skew/customers.parquet")
df_transactions = spark.read.parquet("/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data/data_skew/transactions.parquet")

# Show first few records
df_customers.show(5, truncate=False)
df_transactions.show(5, truncate=False)

# Print schemas
df_customers.printSchema()
df_transactions.printSchema()

# Count records
print("Customers:", df_customers.count())
print("Transactions:", df_transactions.count())


+----------+-------------+---+------+----------+-----+-----------+
|cust_id   |name         |age|gender|birthday  |zip  |city       |
+----------+-------------+---+------+----------+-----+-----------+
|C007YEYTX9|Aaron Abbott |34 |Female|7/13/1991 |97823|boston     |
|C00B971T1J|Aaron Austin |37 |Female|12/16/2004|30332|chicago    |
|C00WRSJF1Q|Aaron Barnes |29 |Female|3/11/1977 |23451|denver     |
|C01AZWQMF3|Aaron Barrett|31 |Male  |7/9/1998  |46613|los_angeles|
|C01BKUFRHA|Aaron Becker |54 |Male  |11/24/1979|40284|san_diego  |
+----------+-------------+---+------+----------+-----+-----------+
only showing top 5 rows
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|cust_id   |start_date|end_date  |txn_id         |date      |year|month|day|expense_type |amt   |city       |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|C0YDPQWPBJ|2010-07-01|2018-12-01|TZ5SMKZY9

In [7]:
df_customers.show(5, False)

+----------+-------------+---+------+----------+-----+-----------+
|cust_id   |name         |age|gender|birthday  |zip  |city       |
+----------+-------------+---+------+----------+-----+-----------+
|C007YEYTX9|Aaron Abbott |34 |Female|7/13/1991 |97823|boston     |
|C00B971T1J|Aaron Austin |37 |Female|12/16/2004|30332|chicago    |
|C00WRSJF1Q|Aaron Barnes |29 |Female|3/11/1977 |23451|denver     |
|C01AZWQMF3|Aaron Barrett|31 |Male  |7/9/1998  |46613|los_angeles|
|C01BKUFRHA|Aaron Becker |54 |Male  |11/24/1979|40284|san_diego  |
+----------+-------------+---+------+----------+-----+-----------+
only showing top 5 rows


In [8]:
df_base = (
    df_customers
    .filter(F.col("city") == "boston")
    .withColumn(
        "customer_group", 
        F.when(
            F.col("age").between(20, 30), 
            F.lit("young") 
        )
        .when(
            F.col("age").between(31, 50), 
            F.lit("mid") 
        )
        .when(
            F.col("age") > 51, 
            F.lit("old") 
        )
        .otherwise(F.lit("kid"))
     )
    .select("cust_id", "name", "age", "gender", "birthday", "zip", "city", "customer_group")
)
#as you can see that the data frame named df_base is going many tarnsformations
#we are doing filtering adding new columns 
# df_base.cache()  for now lets now cache it
df_base.show(5, False)

+----------+--------------+---+------+---------+-----+------+--------------+
|cust_id   |name          |age|gender|birthday |zip  |city  |customer_group|
+----------+--------------+---+------+---------+-----+------+--------------+
|C007YEYTX9|Aaron Abbott  |34 |Female|7/13/1991|97823|boston|mid           |
|C08XAQUY73|Aaron Lambert |54 |Female|11/5/1966|75218|boston|old           |
|C094P1VXF9|Aaron Lindsey |24 |Male  |9/21/1990|29399|boston|young         |
|C097SHE1EF|Aaron Lopez   |22 |Female|4/18/2001|82129|boston|young         |
|C0DTC6436T|Aaron Schwartz|52 |Female|7/9/1962 |57192|boston|old           |
+----------+--------------+---+------+---------+-----+------+--------------+
only showing top 5 rows


In [9]:
df_base.explain()

== Physical Plan ==
*(1) Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, CASE WHEN ((cast(age#2 as bigint) >= 20) AND (cast(age#2 as bigint) <= 30)) THEN young WHEN ((cast(age#2 as bigint) >= 31) AND (cast(age#2 as bigint) <= 50)) THEN mid WHEN (cast(age#2 as bigint) > 51) THEN old ELSE kid END AS customer_group#124]
+- *(1) Filter (isnotnull(city#6) AND (city#6 = boston))
   +- *(1) ColumnarToRow
      +- FileScan parquet [cust_id#0,name#1,age#2,gender#3,birthday#4,zip#5,city#6] Batched: true, DataFilters: [isnotnull(city#6), (city#6 = boston)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data..., PartitionFilters: [], PushedFilters: [IsNotNull(city), EqualTo(city,boston)], ReadSchema: struct<cust_id:string,name:string,age:string,gender:string,birthday:string,zip:string,city:string>




In [11]:
#lets create a dataframe on top of df_base and see what is happening
df1 = (
    df_base
    .withColumn("test_column_1", F.lit("test_column_1"))
    .withColumn("birth_year", F.split("birthday", "/").getItem(2))
)
df1.explain()
df1.show(5,False)

== Physical Plan ==
*(1) Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, CASE WHEN ((cast(age#2 as bigint) >= 20) AND (cast(age#2 as bigint) <= 30)) THEN young WHEN ((cast(age#2 as bigint) >= 31) AND (cast(age#2 as bigint) <= 50)) THEN mid WHEN (cast(age#2 as bigint) > 51) THEN old ELSE kid END AS customer_group#124, test_column_1 AS test_column_1#183, split(birthday#4, /, -1)[2] AS birth_year#184]
+- *(1) Filter (isnotnull(city#6) AND (city#6 = boston))
   +- *(1) ColumnarToRow
      +- FileScan parquet [cust_id#0,name#1,age#2,gender#3,birthday#4,zip#5,city#6] Batched: true, DataFilters: [isnotnull(city#6), (city#6 = boston)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data..., PartitionFilters: [], PushedFilters: [IsNotNull(city), EqualTo(city,boston)], ReadSchema: struct<cust_id:string,name:string,age:string,gender:string,birthday:string,zip:string,city:string>


+----------+--------

* As you can see you df_base transformations are going again from the start

In [12]:
df_base.cache()

DataFrame[cust_id: string, name: string, age: string, gender: string, birthday: string, zip: string, city: string, customer_group: string]

In [13]:
df2 =(
    df_base    .withColumn("test_column_2", F.lit("test_column_2"))

)

In [14]:
df2.explain()

== Physical Plan ==
*(1) Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, customer_group#124, test_column_2 AS test_column_2#256]
+- InMemoryTableScan [age#2, birthday#4, city#6, cust_id#0, customer_group#124, gender#3, name#1, zip#5]
      +- InMemoryRelation [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, customer_group#124], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(1) Project [cust_id#0, name#1, age#2, gender#3, birthday#4, zip#5, city#6, CASE WHEN ((cast(age#2 as bigint) >= 20) AND (cast(age#2 as bigint) <= 30)) THEN young WHEN ((cast(age#2 as bigint) >= 31) AND (cast(age#2 as bigint) <= 50)) THEN mid WHEN (cast(age#2 as bigint) > 51) THEN old ELSE kid END AS customer_group#124]
               +- *(1) Filter (isnotnull(city#6) AND (city#6 = boston))
                  +- *(1) ColumnarToRow
                     +- FileScan parquet [cust_id#0,name#1,age#2,gender#3,birthday#4,zip#5,city#6] Batched: true, DataFilters: [

*  ** INMEMORY TABLE SCAN ** is happening in here
* what that means is we are direclty building our logic for df2 on top of df_base with rescaning and operating the whole thing again

## CACHE VS PERSIST
both are Same the only differnce is storage level you can metion that for persist 

    why that matters 
    the default is memomry and disk deserialized deserilization make it fast access so cpu efficent but not so memery efficient
    disk only cpu efficeint and disk efficent but slow access of the data and the data is serilized when its is stored in the disk
    memoruy only as name suggests it is memory only but fast access less memmory effesicent

### When to use what?
```
Storage Level    Space used  CPU time  In memory  On-disk  Serialized
---------------------------------------------------------------------
MEMORY_ONLY          High        Low       Y          N        N         
MEMORY_ONLY_SER      Low         High      Y          N        Y     
MEMORY_AND_DISK      High        Medium    Some       Some     Some  
MEMORY_AND_DISK_SER  Low         High      Some       Some     Y     
DISK_ONLY            Low         High      N          Y        Y     
```

In [15]:
df_base.unpersist()


DataFrame[cust_id: string, name: string, age: string, gender: string, birthday: string, zip: string, city: string, customer_group: string]

In [16]:
spark.stop()